In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing 

import cv2
import matplotlib.pyplot as plt

from skimage.morphology import extrema
from skimage.color import rgb2gray

# Load cell data

In [ ]:
urothelial_cells = pd.read_pickle("urothelial_cell_toy_data.pkl")
images=np.transpose(urothelial_cells["X"].numpy()*255,(0,2,3,1)).astype(np.uint8)
labels=urothelial_cells["y"]

In [ ]:
idx_image=27
img=images[idx_image]
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')

In [ ]:
label=labels[idx_image]

In [ ]:
img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import data
from skimage.util import img_as_float
from skimage.filters import gabor_kernel

In [ ]:
def power(image, kernel):
    # Normalize images for better comparison.
    image = (image - image.mean()) / image.std()
    return np.sqrt(
        ndi.convolve(image, np.real(kernel), mode='wrap') ** 2
        + ndi.convolve(image, np.imag(kernel), mode='wrap') ** 2
    )


# Plot a selection of the filter bank kernels and their responses.
results = []
kernel_params = []
for theta in np.linspace(0, 1.01, 3):
    theta = theta / 4.0 * np.pi
    for frequency in (0.1, 0.2, 0.3, 0.4):
        kernel = gabor_kernel(frequency, theta=theta)
        params = f"theta={theta * 180 / np.pi},\nfrequency={frequency:.2f}"
        kernel_params.append(params)
        # Save kernel and the power image for each image
        results.append((kernel, power(img_gray, kernel) ))

In [ ]:
kernels=[]
for theta in np.linspace(0, 1.01, 3):
    theta = theta / 4.0 * np.pi
    for frequency in (0.1, 0.2, 0.3, 0.4):
        kernel = gabor_kernel(frequency, theta=theta)
        kernels.append(kernel)


In [ ]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def extract_glcm_features(im_gray, distances=[5], angles=[0], levels=256, properties=['dissimilarity', 'correlation']):
    """
    Extract GLCM features from a grayscale image.

    Parameters:
    - im_gray: Grayscale input image (2D array).
    - distances: List of pixel pair distances for GLCM computation.
    - angles: List of angles (in radians) for GLCM computation.
    - levels: Number of gray levels in the image.
    - properties: List of GLCM properties to compute.

    Returns:
    - glcm_features: N-dimensional image with GLCM features stacked at each pixel location.
    """
    glcm_features = np.zeros(im_gray.shape + (len(distances) * len(angles) * len(properties),), dtype=np.float32)
    
    for i, d in enumerate(distances):
        for j, a in enumerate(angles):
            glcm = graycomatrix(im_gray, distances=[d], angles=[a], levels=levels, symmetric=True, normed=True)
            for k, prop in enumerate(properties):
                prop_values = graycoprops(glcm, prop)
                glcm_features[:,:,i*len(angles)*len(properties) + j*len(properties) + k] = prop_values
                
    return glcm_features

# Example usage:
# Assuming 'im_gray' is your grayscale input image
# glcm_features = extract_glcm_features(im_gray)


In [ ]:
glcm_features=extract_glcm_features(img_gray,distances = [3,5,7], angles = [0,45,90])

In [ ]:
filter_i=2
plt.imshow(kernels[filter_i].astype(float))

In [ ]:
filter_i=3
plt.imshow(results[filter_i][1].astype(float),cmap="gray_r")

In [ ]:
new_feature_img=np.transpose(np.stack([res[1].astype(float) for res in results]),(1,2,0))

In [ ]:
new_feature_img.shape

In [ ]:
all_features = np.concatenate([img,new_feature_img,glcm_features],axis=2)

In [ ]:
all_features.shape

In [ ]:
vector_img = all_features.reshape((-1,all_features.shape[-1]))
# Converts float type
vector_img = np.float32(vector_img)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


vector_img_reduced=PCA(n_components=2).fit_transform(StandardScaler().fit_transform(vector_img))


In [ ]:
plt.scatter(vector_img_reduced[:,0],vector_img_reduced[:,1],c=label.flatten())

In [ ]:
all_features.shape # (height,width,color channel)

In [ ]:
# Converts the dimension into Kx3 matrix
# K = H x W
vector_img = img.reshape((-1,3))
# Converts float type
vector_img = np.float32(vector_img)

In [ ]:
import pandas as pd
pixel_colors=pd.DataFrame(vector_img,columns=["Red","Green","Blue"])#.shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure()
ax=fig.add_subplot(111,projection="3d")
ax.scatter(pixel_colors["Red"],pixel_colors["Green"],pixel_colors["Blue"],c=label.flatten())

In [ ]:
#the below line of code defines the criteria for the algorithm to stop running, 
#which will happen is 100 iterations are run or the epsilon (which is the required accuracy) 
#becomes 85%
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
 
# then perform k-means clustering with number of clusters defined as your desired value
#also random centres are initially choosed for k-means clustering
k = 4
retval, labels, centers = cv2.kmeans(vector_img, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
 
# convert data into 8-bit values
centers = np.uint8(centers)
segmented_data = centers[labels.flatten()]
 
# reshape data into the original image dimensions
segmented_image = segmented_data.reshape((img.shape))

plt.imshow(segmented_image)

In [ ]:
labels[idx_image]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure()
ax=fig.add_subplot(111,projection="3d")
ax.scatter(pixel_colors["Red"],pixel_colors["Green"],pixel_colors["Blue"],c=labels)

In [ ]:
segmented_image_intensity=cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)

In [ ]:
cluster_intensities=sorted(np.unique(segmented_image_intensity))
cluster_intensities[1:-1]

In [ ]:
nuclear_intensity=cluster_intensities[0]
cytoplasm_intensity=cluster_intensities[1:-1]
background_intensity=cluster_intensities[-1]

In [ ]:
nuclear_mask=(segmented_image_intensity==nuclear_intensity)
background_mask=(segmented_image_intensity==background_intensity)
cytoplasm_mask=np.logical_and(segmented_image_intensity!=nuclear_intensity,segmented_image_intensity!=background_intensity)

In [ ]:
print(nuclear_mask)

In [ ]:
plt.imshow(nuclear_mask)

In [ ]:
from scipy.ndimage import label
from scipy.ndimage import binary_fill_holes
plt.subplot(1,2,1)
plt.imshow(img)
plt.axis('off')

nuclear_mask_modified=binary_fill_holes(label(nuclear_mask)[0]==1)
plt.subplot(1,2,2)
plt.imshow(nuclear_mask_modified)
plt.axis('off')

In [ ]:
# Calculate the number of pixels for nuclei and cytoplasm
nuclei_pixels = np.sum(nuclear_mask_modified)
cytoplasm_pixels = np.sum(cytoplasm_mask)

# Compute the ratio
nuclei_cytoplasm_ratio = nuclei_pixels / (nuclei_pixels+cytoplasm_pixels)

print("Nuclei/Cytoplasm Ratio:", nuclei_cytoplasm_ratio)

In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import label as nd_label
from scipy.ndimage import binary_fill_holes

# Assuming `extract_glcm_features` and `res` are defined elsewhere in your code
# Ensure you have the required packages installed, e.g.:
# pip install pandas numpy opencv-python scikit-learn scipy

urothelial_cells = pd.read_pickle("urothelial_cell_toy_data.pkl")
images = np.transpose(urothelial_cells["X"].numpy() * 255, (0, 2, 3, 1)).astype(np.uint8)
labels = urothelial_cells["y"]
nc_true=[]
nc_pred=[]

for i in range(0, 100):
    img = images[i]
    img_label = labels[i]  # Renaming the label variable to img_label
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
    # Ensure extract_glcm_features is defined and works correctly
    glcm_features = extract_glcm_features(img_gray, distances=[3, 5, 7], angles=[0, 45, 90])
    
    # Assuming res is defined elsewhere in your code
    new_feature_img = np.transpose(np.stack([res[1].astype(float) for res in results]), (1, 2, 0))
    
    all_features = np.concatenate([img, new_feature_img, glcm_features], axis=2)
    vector_img = all_features.reshape((-1, all_features.shape[-1]))
    
    # Converts float type
    vector_img = np.float32(vector_img)
    vector_img_reduced = PCA(n_components=2).fit_transform(StandardScaler().fit_transform(vector_img))
    
    vector_img = img.reshape((-1, 3))
    
    # Converts float type
    vector_img = np.float32(vector_img)
    
    # Define the criteria for k-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
    
    # Perform k-means clustering
    k = 4
    retval, cluster_labels, centers = cv2.kmeans(vector_img, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert data into 8-bit values
    centers = np.uint8(centers)
    segmented_data = centers[cluster_labels.flatten()]
    
    # Reshape data into the original image dimensions
    segmented_image = segmented_data.reshape((img.shape))
    segmented_image_intensity = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)
    
    cluster_intensities = sorted(np.unique(segmented_image_intensity))
    
    nuclear_intensity = cluster_intensities[0]
    cytoplasm_intensity = cluster_intensities[1:-1]
    background_intensity = cluster_intensities[-1]
    
    nuclear_mask = (segmented_image_intensity == nuclear_intensity)
    background_mask = (segmented_image_intensity == background_intensity)
    cytoplasm_mask = np.logical_and(segmented_image_intensity != nuclear_intensity, segmented_image_intensity != background_intensity)
#     print(nuclear_mask)
    
    nuclear2_mask_modified = binary_fill_holes(nd_label(nuclear_mask)[0] == 1)
    
    nuclei_pixels = np.sum(nuclear_mask_modified)
    cytoplasm_pixels = np.sum(cytoplasm_mask)

    # Compute the ratio
    nuclei_cytoplasm_ratio = nuclei_pixels / (nuclei_pixels+cytoplasm_pixels)
    nucleus_area=(img_label==2).sum()
    cytoplasm_area=(img_label==1).sum()
    true_nc=nucleus_area/(nucleus_area+cytoplasm_area)
    print(i,": Nuclei/Cytoplasm Ratio:", nuclei_cytoplasm_ratio, "True:", true_nc)
    nc_true.append(true_nc)
    nc_pred.append(nuclei_cytoplasm_ratio)

    


In [ ]:
plt.scatter(nc_pred,nc_true)

In [ ]:
# from scipy.stats import spearmanr
# spearmanr(nc_pred,nc_true)

from sklearn.metrics import classification_report

# print(classification_report(Y_val.numpy().flatten(),y_val_pred_lbls.flatten()))
print(classification_report(nuclear_mask, cytoplasm_mask))

In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import extrema
from skimage.color import rgb2gray
from scipy import ndimage as ndi
from skimage.feature import graycomatrix, graycoprops
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import label as nd_label
from scipy.ndimage import binary_fill_holes
from skimage.filters import gabor_kernel

# Function to extract GLCM features
def extract_glcm_features(im_gray, distances=[5, 7, 9], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, properties=['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'energy']):
    glcm_features = np.zeros(im_gray.shape + (len(distances) * len(angles) * len(properties),), dtype=np.float32)
    for i, d in enumerate(distances):
        for j, a in enumerate(angles):
            glcm = graycomatrix(im_gray, distances=[d], angles=[a], levels=levels, symmetric=True, normed=True)
            for k, prop in enumerate(properties):
                prop_values = graycoprops(glcm, prop)
                glcm_features[:,:,i*len(angles)*len(properties) + j*len(properties) + k] = prop_values
    return glcm_features

# Function to apply Gabor kernels
def apply_gabor_kernels(image, kernels):
    gabor_features = []
    for kernel in kernels:
        filtered = ndi.convolve(image, np.real(kernel), mode='wrap')
        gabor_features.append(filtered)
    return np.stack(gabor_features, axis=-1)

# Define Gabor kernels
def build_gabor_kernels():
    kernels = []
    for theta in np.linspace(0, np.pi, 4):
        for frequency in (0.1, 0.2, 0.3, 0.4):
            kernel = gabor_kernel(frequency, theta=theta)
            kernels.append(kernel)
    return kernels

# Load data
urothelial_cells = pd.read_pickle("urothelial_cell_toy_data.pkl")
images = np.transpose(urothelial_cells["X"].numpy() * 255, (0, 2, 3, 1)).astype(np.uint8)
labels = urothelial_cells["y"]
nc_true = []
nc_pred = []

# Build Gabor kernels
kernels = build_gabor_kernels()

for i in range(0, 3):
    img = images[i]
    img_label = labels[i]  # Renaming the label variable to img_label
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Extract GLCM features
    glcm_features = extract_glcm_features(img_gray, distances=[3, 5, 7], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])
    
    # Apply Gabor kernels
    gabor_features = apply_gabor_kernels(img_gray, kernels)
    
    # Combine all features
    all_features = np.concatenate([img, gabor_features, glcm_features], axis=2)
    vector_img = all_features.reshape((-1, all_features.shape[-1]))
    
    # Standardize and reduce dimensionality
    vector_img = np.float32(vector_img)
    vector_img_reduced = PCA(n_components=2).fit_transform(StandardScaler().fit_transform(vector_img))
    
    # Perform k-means clustering
    vector_img = img.reshape((-1, 3))
    vector_img = np.float32(vector_img)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
    k = 4
    retval, cluster_labels, centers = cv2.kmeans(vector_img, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    segmented_data = centers[cluster_labels.flatten()]
    segmented_image = segmented_data.reshape((img.shape))
    segmented_image_intensity = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)
    
    # Determine intensities
    cluster_intensities = sorted(np.unique(segmented_image_intensity))
    nuclear_intensity = cluster_intensities[0]
    cytoplasm_intensity = cluster_intensities[1:-1]
    background_intensity = cluster_intensities[-1]
    
    # Create masks
    nuclear_mask = (segmented_image_intensity == nuclear_intensity)
    background_mask = (segmented_image_intensity == background_intensity)
    cytoplasm_mask = np.logical_and(segmented_image_intensity != nuclear_intensity, segmented_image_intensity != background_intensity)
    
    # Modify nuclear mask
    nuclear_mask_modified = binary_fill_holes(nd_label(nuclear_mask)[0] == 1)
    
    nuclei_pixels = np.sum(nuclear_mask_modified)
    cytoplasm_pixels = np.sum(cytoplasm_mask)

    # Compute the ratio
    nuclei_cytoplasm_ratio = nuclei_pixels / (nuclei_pixels + cytoplasm_pixels)
    nucleus_area = (img_label == 2).sum()
    cytoplasm_area = (img_label == 1).sum()
    true_nc = nucleus_area / (nucleus_area + cytoplasm_area)
    
    print(i, ": Nuclei/Cytoplasm Ratio:", nuclei_cytoplasm_ratio, "True:", true_nc)
    nc_true.append(true_nc)
    nc_pred.append(nuclei_cytoplasm_ratio)


In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import extrema
from skimage.color import rgb2gray
from scipy import ndimage as ndi
from skimage.feature import graycomatrix, graycoprops
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import label as nd_label
from scipy.ndimage import binary_fill_holes
from skimage.filters import gabor_kernel

# Function to extract GLCM features
def extract_glcm_features(im_gray, distances=[5, 7, 9], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, properties=['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'energy']):
    glcm_features = np.zeros(im_gray.shape + (len(distances) * len(angles) * len(properties),), dtype=np.float32)
    for i, d in enumerate(distances):
        for j, a in enumerate(angles):
            glcm = graycomatrix(im_gray, distances=[d], angles=[a], levels=levels, symmetric=True, normed=True)
            for k, prop in enumerate(properties):
                prop_values = graycoprops(glcm, prop)
                glcm_features[:,:,i*len(angles)*len(properties) + j*len(properties) + k] = prop_values
    return glcm_features

# Function to apply Gabor kernels
def apply_gabor_kernels(image, kernels):
    gabor_features = []
    for kernel in kernels:
        filtered = ndi.convolve(image, np.real(kernel), mode='wrap')
        gabor_features.append(filtered)
    return np.stack(gabor_features, axis=-1)

# Define Gabor kernels
def build_gabor_kernels():
    kernels = []
    for theta in np.linspace(0, np.pi, 4):
        for frequency in (0.1, 0.2, 0.3, 0.4):
            kernel = gabor_kernel(frequency, theta=theta)
            kernels.append(kernel)
    return kernels

# Load data
urothelial_cells = pd.read_pickle("urothelial_cell_toy_data.pkl")
images = np.transpose(urothelial_cells["X"].numpy() * 255, (0, 2, 3, 1)).astype(np.uint8)
labels = urothelial_cells["y"]
nc_true = []
nc_pred = []

# Build Gabor kernels
kernels = build_gabor_kernels()

for i in range(0, 100):
    img = images[i]
    img_label = labels[i]  # Renaming the label variable to img_label
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Extract GLCM features
    glcm_features = extract_glcm_features(img_gray, distances=[3, 5, 7], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])
    
    # Apply Gabor kernels
    gabor_features = apply_gabor_kernels(img_gray, kernels)
    
    # Combine all features
    all_features = np.concatenate([img, gabor_features, glcm_features], axis=2)
    vector_img = all_features.reshape((-1, all_features.shape[-1]))
    
    # Standardize and apply PCA
    vector_img = np.float32(vector_img)
    standardized_features = StandardScaler().fit_transform(vector_img)
    pca = PCA(n_components=2)
    vector_img_reduced = pca.fit_transform(standardized_features)
    
    # Perform k-means clustering
    k = 4
    retval, cluster_labels, centers = cv2.kmeans(vector_img_reduced, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    segmented_data = centers[cluster_labels.flatten()]
    segmented_image = segmented_data.reshape((img.shape[:2]))  # Only reshape to 2D for intensity
    
    # Convert to intensity image for further processing
    segmented_image_intensity = segmented_image
    
    # Determine intensities
    cluster_intensities = sorted(np.unique(segmented_image_intensity))
    nuclear_intensity = cluster_intensities[0]
    cytoplasm_intensity = cluster_intensities[1:-1]
    background_intensity = cluster_intensities[-1]
    
    # Create masks
    nuclear_mask = (segmented_image_intensity == nuclear_intensity)
    background_mask = (segmented_image_intensity == background_intensity)
    cytoplasm_mask = np.logical_and(segmented_image_intensity != nuclear_intensity, segmented_image_intensity != background_intensity)
    
    # Modify nuclear mask
    nuclear_mask_modified = binary_fill_holes(nd_label(nuclear_mask)[0] == 1)
    
    nuclei_pixels = np.sum(nuclear_mask_modified)
    cytoplasm_pixels = np.sum(cytoplasm_mask)

    # Compute the ratio
    nuclei_cytoplasm_ratio = nuclei_pixels / (nuclei_pixels + cytoplasm_pixels)
    nucleus_area = (img_label == 2).sum()
    cytoplasm_area = (img_label == 1).sum()
    true_nc = nucleus_area / (nucleus_area + cytoplasm_area)
    
    print(i, ": Nuclei/Cytoplasm Ratio:", nuclei_cytoplasm_ratio, "True:", true_nc)
    nc_true.append(true_nc)
    nc_pred.append(nuclei_cytoplasm_ratio)



In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import label as nd_label
from scipy.ndimage import binary_fill_holes
from skimage.feature import graycomatrix, graycoprops

# Function to extract GLCM features
def extract_glcm_features(im_gray, distances=[3, 5, 7], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, properties=['dissimilarity', 'correlation']):
    glcm_features = np.zeros(im_gray.shape + (len(distances) * len(angles) * len(properties),), dtype=np.float32)
    for i, d in enumerate(distances):
        for j, a in enumerate(angles):
            glcm = graycomatrix(im_gray, distances=[d], angles=[a], levels=levels, symmetric=True, normed=True)
            for k, prop in enumerate(properties):
                prop_values = graycoprops(glcm, prop)
                glcm_features[:,:,i*len(angles)*len(properties) + j*len(properties) + k] = prop_values
    return glcm_features

# Load data
urothelial_cells = pd.read_pickle("urothelial_cell_toy_data.pkl")
images = np.transpose(urothelial_cells["X"].numpy() * 255, (0, 2, 3, 1)).astype(np.uint8)
labels = urothelial_cells["y"]
nc_true = []
nc_pred = []

for i in range(0, 100):
    img = images[i]
    img_label = labels[i]  # Renaming the label variable to img_label
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Extract GLCM features
    glcm_features = extract_glcm_features(img_gray, distances=[3, 5, 7], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])
    
    # Combine RGB and GLCM features
    all_features = np.concatenate([img, glcm_features], axis=2)
    vector_img = all_features.reshape((-1, all_features.shape[-1]))
    
    # Standardize and apply PCA
    vector_img = np.float32(vector_img)
    standardized_features = StandardScaler().fit_transform(vector_img)
    pca = PCA(n_components=2)
    vector_img_reduced = pca.fit_transform(standardized_features)
    
    # Perform k-means clustering
    k = 4
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)
    retval, cluster_labels, centers = cv2.kmeans(vector_img_reduced, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    segmented_data = centers[cluster_labels.flatten()]
    segmented_image = segmented_data.reshape((img.shape[:2]))  # Only reshape to 2D for intensity
    
    # Convert to intensity image for further processing
    segmented_image_intensity = segmented_image
    
    # Determine intensities
    cluster_intensities = sorted(np.unique(segmented_image_intensity))
    nuclear_intensity = cluster_intensities[0]
    cytoplasm_intensity = cluster_intensities[1:-1]
    background_intensity = cluster_intensities[-1]
    
    # Create masks
    nuclear_mask = (segmented_image_intensity == nuclear_intensity)
    background_mask = (segmented_image_intensity == background_intensity)
    cytoplasm_mask = np.logical_and(segmented_image_intensity != nuclear_intensity, segmented_image_intensity != background_intensity)
    
    # Modify nuclear mask
    nuclear_mask_modified = binary_fill_holes(nd_label(nuclear_mask)[0] == 1)
    
    nuclei_pixels = np.sum(nuclear_mask_modified)
    cytoplasm_pixels = np.sum(cytoplasm_mask)

    # Compute the ratio
    nuclei_cytoplasm_ratio = nuclei_pixels / (nuclei_pixels + cytoplasm_pixels)
    nucleus_area = (img_label == 2).sum()
    cytoplasm_area = (img_label == 1).sum()
    true_nc = nucleus_area / (nucleus_area + cytoplasm_area)
    
    print(i, ": Nuclei/Cytoplasm Ratio:", nuclei_cytoplasm_ratio, "True:", true_nc)
    nc_true.append(true_nc)
    nc_pred.append(nuclei_cytoplasm_ratio)
